In [ ]:
%reset -f

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

import transformers
import torch
import os
import platform

In [ ]:
#chdir
if platform.system() == 'Darwin':

    print("GPTQ is designed for NV cards")

elif platform.system() == 'Linux':
    
    os.chdir("/home/ning/Data/Dropbox/Working_Directory/NLP/langchain_llama2")
    
    print("=====================================================")
    print("Linux detected; PopOS recommended for GPU computing.")
    print("the current wd is :")
    print(os.getcwd())
    print("=====================================================")

else :
    
    print("I hate Windows")

In [ ]:
with open('data/feature/test1_abs.txt', 'r+', encoding="utf-8") as file:
    context = file.read()
# context = context[:726]

In [ ]:
# model_name_or_path = "TheBloke/Llama-2-7B-GPTQ"
# model_basename = "gptq_model-4bit-128g"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained("./model/Llama-2-7B-GPTQ", use_fast=True)

model = AutoGPTQForCausalLM.from_quantized("./model/Llama-2-7B-GPTQ",
                                           model_basename="gptq_model-4bit-128g",
                                           use_safetensors=True,
                                           trust_remote_code=True,
                                           device="cuda:0",
                                           use_triton=use_triton,
                                           quantize_config=None)

prompt = "who is the victim in the following article ? "
prompt_template=f'''{prompt} : {context}
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

# Prevent printing spurious transformers error when using pipeline with AutoGPTQ
logging.set_verbosity(logging.CRITICAL)

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

In [ ]:
result = pipe(prompt_template)

In [ ]:
result[0]